---
title: "Stock Market Analysis"

description: "Extract SP500 tickers data and analyze it for finding signals to trade"
author: "Aakash Basnet"
date: "2024/01/15"
page-layout: full
categories:
  - python
  - stock
format:
  html:
    code-fold: true
jupyter: python3
---


!["Stock Analysis(Generated by Imagen3)"](stock_analysis.png)

# Installation
On your terminal run the following command to install yfinance module.
```
pip install yfinance
```

# Extract Company in SP500 
Firstly, lets extract the SP500 tickers from wikipedia table.

In [169]:
import pandas as pd
import yfinance as yf

from pprint import pprint

start_date = '2020-01-01'
end_date = '2025-01-15'

tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers['Symbol'] = tickers['Symbol'].apply(lambda x: x.strip())
tickers.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


# Download history data for SP500 companies
For each symbol in SP500, lets download daily historical data.

In [170]:

data = yf.download(tickers.Symbol.to_list(),
                   start=start_date,
                   end=end_date,
                   interval='1D',
                   auto_adjust=True)
data.tail()

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2020-01-01 -> 2025-01-15)')


Price      Adj Close             Close                                      \
Ticker          BF.B BRK.B           A        AAPL        ABBV        ABNB   
Date                                                                         
2025-01-07       NaN   NaN  137.410004  242.210007  179.529999  131.289993   
2025-01-08       NaN   NaN  137.000000  242.699997  178.500000  130.800003   
2025-01-10       NaN   NaN  137.470001  236.850006  175.169998  129.630005   
2025-01-13       NaN   NaN  141.949997  234.399994  176.740005  128.850006   
2025-01-14       NaN   NaN  143.429993  233.279999  175.550003  127.599998   

Price                                                      ...  Volume  \
Ticker             ABT       ACGL         ACN        ADBE  ...     WTW   
Date                                                       ...           
2025-01-07  113.400002  92.250000  356.390015  422.630005  ...  402500   
2025-01-08  114.250000  92.660004  357.730011  419.579987  ...  655000   
2025-01-10  112.309998  90.169998  349.790009  405.920013  ...  594700   
2025-01-13  113.190002  90.830002  349.140015  408.500000  ...  459400   
2025-01-14  113.019997  91.989998  348.989990  412.709991  ...  441700   

Price                                                                       \
Ticker           WY     WYNN      XEL       XOM      XYL      YUM      ZBH   
Date                                                                         
2025-01-07  2588200  2195300  3017800  12625900  1232400  2103000  1649100   
2025-01-08  3939100  1862300  3714700  17858100  1274500  2025400  2385600   
2025-01-10  3529500  2655900  5441000  19304500  1334500  2555000  2709300   
2025-01-13  4299500  1850500  3114600  17073400  1154300  2163100  1565800   
2025-01-14  4151400  2773800  5952800  11187700  2137000  1793600  1457100   

Price                        
Ticker        ZBRA      ZTS  
Date                         
2025-01-07  353800  2488500  
2025-01-08  413600  2353200  
2025-01-10  460000  3179500  
2025-01-13  505200  2306100  
2025-01-14  414500  3608200  

[5 rows x 2517 columns]

# Calculate Rolling Average 
Now, The 100 day rolling average is calculated and compared with latest closing price.
Then, the  data where the 100 days rolling average is less than latest closing price is filtered

In [185]:
window_sizes = [5, 10, 30, 60, 100, 200] # indays
results = []
for ticker in data.columns.get_level_values(1).unique():

    ticker_data = data['Close'][ticker]
    row = [ticker,ticker_data.index[-1],ticker_data.iloc[-1].round(2)]
    columns = ['Ticker','Date','Close']

    row = row + [(ticker_data.tail(window_size).sum()/window_size).round(2) for window_size in window_sizes]
    columns = columns + [f'{window_size}D_MA_Close' for window_size in window_sizes]
    results.append(row)
    
rolling_avg_df = pd.DataFrame(results, columns= columns)
rolling_avg_df.dropna(inplace=True)
print(rolling_avg_df.shape)
rolling_avg_df.head(10)

(501, 9)


,Ticker,Date,Close,5D_MA_Close,10D_MA_Close,30D_MA_Close,60D_MA_Close,100D_MA_Close,200D_MA_Close
2,A,2025-01-14,143.43,139.45,137.13,137.52,135.46,137.74,137.47
3,AAPL,2025-01-14,233.28,237.89,242.43,246.49,237.75,232.84,215.77
4,ABBV,2025-01-14,175.55,177.10,178.02,177.04,181.03,185.97,177.18
5,ABNB,2025-01-14,127.60,129.63,131.38,133.79,134.93,130.46,138.81
6,ABT,2025-01-14,113.02,113.23,113.24,114.23,115.39,114.81,110.00
7,ACGL,2025-01-14,91.99,91.58,91.57,93.38,95.67,99.87,96.29
8,ACN,2025-01-14,348.99,352.41,352.03,357.34,358.54,353.79,331.33
9,ADBE,2025-01-14,412.71,413.87,426.26,468.59,484.06,504.11,505.86
10,ADI,2025-01-14,214.65,215.26,214.60,215.53,218.12,221.07,218.18
11,ADM,2025-01-14,51.19,50.73,50.42,51.31,52.58,55.17,57.66


## Ticker where rolling average is less than closing price

In [186]:
filtered_df = rolling_avg_df[rolling_avg_df['Close'] < rolling_avg_df['100D_MA_Close']]
print(filtered_df.shape)
filtered_df.tail(20)

(301, 9)


,Ticker,Date,Close,5D_MA_Close,10D_MA_Close,30D_MA_Close,60D_MA_Close,100D_MA_Close,200D_MA_Close
471,VMC,2025-01-14,260.07,255.01,256.05,268.87,271.83,260.46,257.85
472,VRSK,2025-01-14,273.49,274.31,274.54,280.36,279.25,275.03,264.28
474,VRTX,2025-01-14,411.66,411.00,407.24,433.34,455.10,462.61,457.21
476,VTR,2025-01-14,58.10,57.77,57.90,59.30,61.70,61.87,55.30
477,VTRS,2025-01-14,11.72,11.83,12.06,12.47,12.36,12.01,11.48
478,VZ,2025-01-14,38.28,38.14,38.69,40.14,40.72,41.23,39.95
483,WDAY,2025-01-14,246.03,249.22,252.50,262.94,257.74,253.96,244.55
484,WDC,2025-01-14,62.74,63.30,62.74,65.35,66.18,65.75,68.67
485,WEC,2025-01-14,94.58,93.96,93.80,95.38,96.48,95.46,88.05
486,WELL,2025-01-14,126.87,125.62,125.36,127.89,131.26,128.62,114.90


# Top holding in SP500

In [182]:
spy = yf.Ticker('SPY').funds_data
spy.top_holdings


,Name,Holding Percent
Symbol,,
AAPL,Apple Inc,0.075770
NVDA,NVIDIA Corp,0.065938
MSFT,Microsoft Corp,0.062729
AMZN,Amazon.com Inc,0.041097
META,Meta Platforms Inc Class A,0.025549
TSLA,Tesla Inc,0.022575
GOOGL,Alphabet Inc Class A,0.022140
AVGO,Broadcom Inc,0.021675
GOOG,Alphabet Inc Class C,0.018142


In [3]:


import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"



In [4]:

import plotly.express as px
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", 
                 color="species", 
                 marginal_y="violin", marginal_x="box", 
                 trendline="ols", template="simple_white")
fig.show()